In [43]:
import numpy as np
from fenics import *
from skopt.sampler import Lhs
from skopt.space import Space

In [44]:
# Use Latin Hypercube Sampling to generate samples for the parameters alpha, beta and gamma
space_lhs = Space([(1., 10.), (0, np.pi/2), (0, np.pi/2)]) # define the space of the parameters for alpha, beta and gamma
lhs = Lhs(lhs_type="classic", criterion="maximin", iterations=1000)

n_echantillons = 100
lhs_samples = np.array(lhs.generate(space_lhs.dimensions, n_echantillons))

In [45]:
def boundary_R(x, on_boundary):
    return on_boundary and near(x[0], 1.0, 1E-14)

def boundary_R_barre(x, on_boundary):
    return on_boundary and not(near(x[0], 1.0, 1E-14))

u_R = Expression('cos(2 * pi * x[1])', degree=2)
u_R_barre = Constant(1.0)

In [46]:
import scipy.io as sio

nx, ny = 9, 9
mesh = UnitSquareMesh(nx, ny)
V = FunctionSpace(mesh, 'P', 1)
bc_R = DirichletBC(V, u_R, boundary_R)
bc_R_barre = DirichletBC(V, u_R_barre, boundary_R_barre)
bcs = [bc_R_barre, bc_R]
source_term_str = 'alpha * pow(pi, 2) * (x[0]*x[0] + x[1]*x[1]) * sin(pi*(x[0] + beta)*(x[1] + gamma))'

source_term_list = []
u_list = []

for alpha, beta, gamma in lhs_samples:

    source_term = Expression(source_term_str, alpha=alpha, beta=beta, gamma=gamma, degree=2)
    source_term = project(source_term, V)
    u = TrialFunction(V)
    v = TestFunction(V)
    a = dot(grad(u), grad(v)) * dx
    L = source_term * v * dx 

    u = Function(V)
    solve(a == L, u, bcs)

    source_term_values, u_values = source_term.vector().get_local(), u.vector().get_local()
    source_term_values, u_values = source_term_values.reshape((nx+1, ny+1)), u_values.reshape((nx+1, ny+1))

    source_term_list.append(source_term_values)
    u_list.append(u_values)

scipy.io.savemat('source_term.mat', {'source_term': source_term_list})
scipy.io.savemat('u.mat', {'u': u_list})
    

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational p